## 시나리오
- 인터파크 투어 사이트(main_url) 에서 여행지(keyword)를 입력 후 검색 -> 잠시후 -> 결과
- 로그인시 pc 웹 사이트에서 처리가 어려율 경우 -> 모바일 로그인 진입
- 모듈 가져오기
    - pip install selenium

- 사전에 필요한 정보를 로드 => 디비 혹은 쉘, 배치 파일에서 인자로 받아서 세팅
- 드라이버 로드
- 사이트 접속(get)
- 검색창을 찾아서 검색어 입력
- 검색 버튼 클릭
- 잠시 대기 -> 페이가 로드되고 나서 즉각적으로 데이터를 획득하는 행위는 자제

In [3]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
import nbimporter

# 명시적 대기를 위해
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
from bs4 import BeautifulSoup as bs
from Tour import TourInfo
from DbMgr import DBHelper as Db

        
# 접속하려는 url 지정 
# 사전에 필요한 정보를 로드 => 디비 혹은 쉘, 배치 파일에서 인자로 받아서 세팅
# keyword 는 선수단어
db = Db()
main_url = 'http://tour.interpark.com/'
keyword = '미국'
# 상품 정보를 담는 리스트 (TourInfo 리스트)
tour_list = []

# 드라이버 로드
driver = wd.Chrome(executable_path='chromedriver.exe')

#차후 -> 옵션 부여하여(프록시, 에이전트 조작(pc/모바일), 이미지 배제)
#크롤링을 오래돌리면 => 임시파일들이 쌓인다!! => 템포 파일 삭제

driver.get(main_url)

#검색창을 찾아서 검색어 입력
# id: SearchGNBText
# 수정할 경우 => 뒤에 내용이 붙어버림(게시글 수정에서 문제가 종종 발생) => .claer() ->send_keys 
driver.find_element_by_id('SearchGNBText').send_keys(keyword)

# 검색 버튼 클릭 
driver.find_element_by_css_selector('button.search-btn').click()

# 잠시 대기 -> 페이가 로드되고 나서 즉각적으로 데이터를 획득하는 행위는 자제
# 참고 : https://selenium-python.readthedocs.io/waits.html
# 명시적 대기(explicit wait) => 특정 요소가 로케이트(발견될 떄 까지) 대기
try:
    element = WebDriverWait(driver, 10).until(
        # 지정한 한개 요소가 올라오면 웨이트 종료
        EC.presence_of_element_located((By.CLASS_NAME, 'oTravelBox'))
    )
except Exception as e:
    print('explicit wait 오류 발생', e)
    
# 암묵적 대기(implicit wait) => DOM 이 다 로드 될때까지 대기하고 먼저 로드되면 바로 진행
# 요소를 찾을 특정 시간(10)동안 DOM 풀링을 지시. 예를 들어 10초 이내에 발생하더라도 바로 진행 
driver.implicitly_wait( 10 )
# 절대적 대기 => time.sleep(10) : 무조건 기다려, 10초 이후 진행  => 클라우드 페어(디도스 방어 솔루션)

# '더 보기' 눌러서 => 게시판으로 진입 
# .oTravelBox > ul > li.moreBtnWrap > button
driver.find_element_by_css_selector('.oTravelBox >.boxList >.moreBtnWrap > .moreBtn').click()

# issue1) 상세 페이지 들고 올때 해당 창에서 열리는지 vs 새로운 창에서 열리는지도 중요 
# issue2) 로그인이 필요한 페이지를 1만개 가지고 온다면 session 이 끊김. 지속적으로 login - logout 필요
# 게시판에서 데이터를 가져올때
# 데이터가 많으면 세션(혹시 로그인을 해서 접근되는 사이트일 경우)관리
# 특정 단위별로 로그아웃 로그인 계속 시도
# 특정 게시물이 사라질 경우 => 팝업 발생(없는... )=> 팝업 처리 검토 과정 필요(예외처리)
# 게시판 스캔시 => 임계점을 모름 ( 수집이 안되는 순간에 대한 고민)
# 게시판 스캔 => 메타 정보 획득(BFS) => loop를 돌려서 일괄적으로 방문 접근 처리

# searchModule.SetCategoryList(1, '') 스크립트 실행 => 스크립트: URL 이 안변함, 즉 스크립트 처리
# 16 은 임시값, 게시물을 넘어갔을때 현상을 확인하고자
for page in range(1,2): #16):
    try:
        # 자바스크립트 구동하기
        driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
        time.sleep(2)
        print("%s 페이지 이동" % page)
        #################################
        # 여러 사이트에서 정보를 수집할 경우: DB 테이블에 공통 정보 정의 단계 필요
        # 상품명, 코멘트, 기간1, 기간2, 가격, 평점, 썸네일, 링크(상품상세정보)
        # full selector : body > div.container > div > div > div.panelZone > div.oTravelBox > ul > li:nth-child(1)
        # boxItems = driver.find_elements_by_css_selector('body > div.container > div > div > div.panelZone > div.oTravelBox > ul > li')
        boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')


        # 상품 하나 하나 접근
        for li in boxItems:
            # 이미지의 링크값을 사용할 것인가? => 이미지에 의존적
            # 직접 다운로드 해서 우리 서버에 업로드(ftp) 할 것인가? 
            print('썸네일',li.find_element_by_css_selector('img').get_attribute('src'))
            print('링크', li.find_element_by_css_selector('a').get_attribute('onclick'))           
            print('상품명', li.find_element_by_css_selector('h5.proTit').text)
            print('코멘트', li.find_element_by_css_selector('.proSub').text)
            print('가격', li.find_element_by_css_selector('.proPrice').text)
            area = ''
            for info in li.find_elements_by_css_selector('.info-row .proInfo'):
                print( info.text )          
            print('='*100)
            # 데이터 모음
            # li.find_elements_by_css_selector('.info-row .proInfo')[1].text
            # 데이터가 부족하거나 없을 수도 있으므로 직접 인덱스로 표현은 위험성이 있음 
            obj = TourInfo( 
                li.find_element_by_css_selector('h5.proTit').text,
                li.find_element_by_css_selector('.proPrice').text,
                li.find_elements_by_css_selector('.info-row .proInfo')[1].text,
                li.find_element_by_css_selector('a').get_attribute('onclick'),
                li.find_element_by_css_selector('img').get_attribute('src')
            )
            tour_list.append(obj)
    except Exception as e1:
        print('e1오류', e1)
        
print( 'tour_list 개수', len(tour_list))
# 수집한 정보 개수를 루프 => 페이지 방문 => 콘텐츠 획득(상품상세정보) , selenium -> beautifulsoup => DB
 
for tour in tour_list:
    # tour => tourInfo
    print( type(tour))
    # 링크 데이터에서 실데이터 획득
    # 분해
    arr = tour.link.split(',')
    if arr:
        # 대체
        link = arr[0].replace('searchModule.OnClickDetail(','')
        # 슬라이싱 => 앞에 ', 뒤에 ' 제거
        detail_url = link[1:-1]
        # 상세 페이지 이동 : URL 값이 완성된 형태인지 확인 (http~)
        driver.get(detail_url)
        time.sleep(2)
        
        # 현재 페이지를 BeautifulSoup 의 DOM으로 구성
        soup = bs( driver.page_source, 'html.parser')
        # 현제 상세 정보 페이지에서 스케줄 정보 획득
        data = soup.select('.tip-cover')
#         print( type(data), len(data))
        
        # DB 입력 => pip install pymysql 
        content_final = ''
        for c in data[0].contents:
            content_final += str(c)
            
        import re
        content_final   = re.sub("'", '"', content_final)
        content_final   = re.sub(re.compile(r'\r\n|\r|\n|\n\r+'), '', content_final)
        print("제목: " + tour.title, "가격: " + tour.price, "기간: " + tour.area, "검색어: " + keyword)
        print('content final', content_final)

        db.db_insertCrawlingData(
                tour.title,
                tour.price[:-1],
                tour.area.replace('출발 가능 기간 : ', ''),
                content_final,
                keyword
            )
#         db.db_insertKeyword(keyword)
        
# 종료
driver.close()
driver.quit()
import sys
sys.exit()

1 페이지 이동
썸네일 http://tourimage.interpark.com/product/tour/00161/A20/280/A2012593_1_567.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A2012593','')
상품명 ★선착순특가[캘리포니아 미서부]3대캐년+LA/라스베가스/샌프란시스코+몬트레이17마일 10일
코멘트 ★★특가 프로모션 진행 중!!
가격 1,590,000 원~
여행 기간 : 8박10일인천-로스앤젤레스-바스토우-라스베가스-샌프란시스코-인천
출발 가능 기간 : 2018.05.21~2018.09.24
평점 10
1개의 상품평
썸네일 http://tourimage.interpark.com/product/tour/00161/A20/280/A2012593_1_567.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A2012593','')
상품명 ★선착순특가[캘리포니아 미서부]3대캐년+LA/라스베가스/샌프란시스코+몬트레이17마일 10일
코멘트 ★★특가 프로모션 진행 중!!
가격 1,590,000 원~
여행 기간 : 8박10일인천-로스앤젤레스-바스토우-라스베가스-샌프란시스코-인천
출발 가능 기간 : 2018.05.21~2018.09.24
평점 10
1개의 상품평
썸네일 http://tourimage.interpark.com/product/tour/00161/A20/280/A2012162_5_880.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A2012162','http://cc.toast.com/cc?a=kw8LaYJ2TBEepMn&m=0&sid=x1fqoweqsrhaiavueygfoxab&pgid=02ec8c2bd56e965

출발 가능 기간 : 2018.06.17~2018.09.16
평점 0
0개의 상품평
tour_list 개수 12
<class 'Tour.TourInfo'>
제목: ★선착순특가[캘리포니아 미서부]3대캐년+LA/라스베가스/샌프란시스코+몬트레이17마일 10일 가격: 1,590,000 원~ 기간: 출발 가능 기간 : 2018.05.21~2018.09.24 검색어: 미국
content final  인터파크 혜택 <div class="default-section special" id="divGoodsSpecialInfo"><h3>인터파크 혜택</h3><span style="FONT-FAMILY: 돋움체,dotumche,applegothic; COLOR: #000000; FONT-SIZE: 9pt"><span style="FONT-FAMILY: 돋움체,dotumche,applegothic; COLOR: #000000; FONT-SIZE: 9pt"><span style="FONT-FAMILY: 돋움체,dotumche,applegothic; COLOR: #000000; FONT-SIZE: 9pt"><span style="FONT-FAMILY: 돋움체,dotumche,applegothic; COLOR: #000000; FONT-SIZE: 9pt"><span style="FONT-FAMILY: 돋움체,dotumche,applegothic; COLOR: #000000; FONT-SIZE: 9pt"><span style="FONT-FAMILY: 돋움체,dotumche,applegothic; COLOR: #000000; FONT-SIZE: 9pt"><span style="FONT-FAMILY: 돋움체,dotumche,applegothic; COLOR: #000000; FONT-SIZE: 9pt"><span style="FONT-FAMILY: 돋움체,dotumche,applegothic; COLOR: #000000; FONT-SIZE: 9pt"><span style="FONT-FAMILY

제목: [미국 첫걸음]미서부 3대캐년 7일 가격: 1,532,800 원~ 기간: 출발 가능 기간 : 2018.06.17~2018.09.24 검색어: 미국
content final  인터파크 혜택 <div class="default-section special" id="divGoodsSpecialInfo"><h3>인터파크 혜택</h3><p><span style="FONT-FAMILY: 돋움,dotum; COLOR: #000000; FONT-SIZE: 9pt">● 미서부 (로스엔젤레스+라스베가스+자이언캐년+그랜드캐년+브라이스캐년)<br/>- 인간이 창조한 세계 최대의 환락 엔터테인먼트 도시 라스베가스!!<br/>- 미서부 3대 캐년 중 가장 다양한 동식물이 살고 있는 자이언캐년 관광!!<br/>- 아름다운 첨탑의 향연 브라이스캐년 국립공원!!<br/>- 사진작가들에게 최고의 포토존으로 꼽히는 명소 엔텔로프 캐년!!</span></p><p><span style="FONT-FAMILY: 돋움,dotum; COLOR: #000000; FONT-SIZE: 9pt"> <br/>※ 현지 합류 상품 안내 <br/>해당 상품 미국현지에서 모객한 인원과 한국 출발인원이 함께 투어 합니다. <br/>해당 상품은 항공사별, 출발지별, 예약시점별 행사인원의 가격이 모두 다를 수 있습니다. <br/>해당 상품은 인원에 따라 인원별 차량이 배정되며 일반적으로 최대 56명의 행사인원이 함께 행사합니다. <br/>해당 상품은 미팅, 샌딩, 행사 가이드가 일정중 변경 될 수 있습니다. <br/>본 상품의 예약과 취소는 국외여행 특별약관이 적용됨을 안내드리며, 특별약관 적용시 표준약관보다 높은 취소수수료가 부과될 수 있습니다.</span></p></div> 인터파크 혜택  포함사항 + 불포함사항 <div class="default-section tip" id="divGoodsIncludeAndExcludeInfo"><div class="includ" id="divGoodsIncludeInfo">

제목: 캐나다 & 미국 동부일주 [6박7일] 가격: 1,012,100 원~ 기간: 출발 가능 기간 : 2018.06.21~2018.10.04 검색어: 미국
content final  인터파크 혜택 <div class="default-section special" id="divGoodsSpecialInfo"><h3>인터파크 혜택</h3><p style="TEXT-ALIGN: center"><strong>※ 2010년 "파란" 개발상품 ※</strong></p><p> </p><p>1. <strong>시장점유율 90%</strong>(지난 7년간 매년 90%이상)가 입증하는 <strong>‘파란’퀘벡 2박3일</strong>과</p><p>   <strong>시장점유율 97%</strong>(2017년 대형버스 출발기준)가 입증하는 ‘<strong>파란’뉴욕+워싱턴 3박4일</strong>을</p><p>   한번에 즐기시는 상품입니다.</p><p> </p><p>2. 북미(캐나다와 미국)의 핵심 관광지인 <strong>퀘벡 + 몬트리얼 + 오타와 + 뉴욕 + 워싱턴</strong>의 관광명소들을</p><p>   짜임새있게 관광하시는, ‘파란’이 자부하는 아주 잘 짜여진 일정입니다.</p><p> </p><p>3. <strong>**호텔의 위치 &amp; 규모 &amp; 수준 - 이보다 더 좋을수는 없습니다!!**</strong></p><p><strong>   퀘벡과 몬트리얼 다운타운 숙박으로 저녁시간 100% 활용가능!!</strong></p><p> </p><p>  <strong> *몬트리얼</strong> 다운타운 초특급 <strong>"하얏트"</strong>호텔(급)-<strong>"파란"독점! 계약호텔</strong>(595객실규모)</p><p>   (다운타운위치+대형쇼핑몰+최신식 증기/건식 사우나+전망좋은수영장+헬스클럽+초대형규모=최고의 호텔)</p><p>   +클럽(몬트리얼에서 제일 유명한)</p><p> </p><p>  <strong> *퀘벡 </strong>

제목: [New York] 대한항공 직항, 맨하튼중심 준특급 호텔 5박7일 가격: 2,025,200 원~ 기간: 출발 가능 기간 : 2018.06.16~2018.10.31 검색어: 미국
content final  인터파크 혜택 <div class="default-section special" id="divGoodsSpecialInfo"><h3>인터파크 혜택</h3><p></p><p><strong><span style="BACKGROUND-COLOR: #ffef00; FONT-SIZE: 14pt"></span></strong></p><p><strong><span style="BACKGROUND-COLOR: #ffef00; FONT-SIZE: 14pt"><img border="0" height="190" rheight="190" rwidth="322" src="http://tourimage.interpark.com/BBS/Tour/FckUpload/201612/6361654826970909270.jpg" style="WIDTH: 322px; HEIGHT: 190px; rwidth: 322px; rheight: 190px" width="322"/><img border="0" height="189" rheight="189" rwidth="331" src="http://tourimage.interpark.com/BBS/Tour/FckUpload/201612/6361654829666644990.jpg" style="WIDTH: 331px; HEIGHT: 189px; rwidth: 331px; rheight: 189px" width="331"/></span></strong></p><p><strong><span style="BACKGROUND-COLOR: #ffef00; FONT-SIZE: 14pt"><img border="0" height="218" rheight="218" rwidth="327" src="http://tourimage.interpark.com/BBS/To

제목: [럭셔리 뉴욕]대한항공 비즈니스 + 맨하튼중심 특급호텔 자유7일 가격: 6,025,200 원~ 기간: 출발 가능 기간 : 2018.06.16~2018.10.31 검색어: 미국
content final  인터파크 혜택 <div class="default-section special" id="divGoodsSpecialInfo"><h3>인터파크 혜택</h3><span style="COLOR: #ff6c00"><span style="COLOR: #00b0a2"><span id="husky_bookmark_start_1393832562697"><span><span><span><span style="COLOR: #ff6c00"><span style="COLOR: #00b0a2"><span id="husky_bookmark_start_1393832562697"><span><span><span><span style="COLOR: #ff6c00"><span style="COLOR: #00b0a2"><span id="husky_bookmark_start_1393832562697"><span><span><span><p><span style="COLOR: rgb(0,0,0)"><span><span style="COLOR: rgb(255,0,0)"><strong>※ 인터파크투어 예약 고객을 위한 혜택 ※ </strong></span></span></span></p><p><span style="COLOR: rgb(0,0,0)"><span><span style="COLOR: rgb(255,0,0)"><strong></strong></span></span></span> <span style="COLOR: rgb(0,0,0)"><span><span style="COLOR: rgb(255,0,0)"><span style="COLOR: rgb(255,108,0)"><span style="COLOR: rgb(0,176,162)"><span><span><span><span><span><s

제목: ◆ 하와이 4개섬 ◆ 호놀룰루 1박 + 크루즈 7박 가격: 4,390,000 원~ 기간: 출발 가능 기간 : 2018.08.25~2018.12.01 검색어: 미국
content final  인터파크 혜택 <div class="default-section special" id="divGoodsSpecialInfo"><h3>인터파크 혜택</h3><p><strong><span style="FONT-FAMILY: 돋움,dotum; COLOR: #000000">※ 국적기 <span style="COLOR: #000000"><span style="COLOR: #ef007c">아시아나항공</span> </span>이용</span></strong></p><p><strong><span style="FONT-FAMILY: 돋움,dotum; COLOR: #000000"><span style="FONT-FAMILY: 돋움,dotum; COLOR: #000000">※ </span>하와이 크루즈를 대표하는 8만톤급 <span style="COLOR: #ef007c">노르웨지안크루즈 프라이드 오브 아메리카호</span> 이용</span></strong></p><p><strong><span style="FONT-FAMILY: 돋움,dotum; COLOR: #000000"><span style="FONT-FAMILY: 돋움,dotum; COLOR: #000000">※ </span>15명 이상 출발시 <span style="COLOR: #ef007c">전 일정 전문 인솔자 동행</span></span></strong></p><p><strong><span style="FONT-FAMILY: 돋움,dotum; COLOR: #464646"><span style="FONT-FAMILY: 돋움,dotum; COLOR: #000000"><span style="FONT-FAMILY: 돋움,dotum; COLOR: #000000">※ <font color="#3a32c3">크루즈 용품 제공~ (선실

제목: ★에메랄드프린세스호★ 알래스카 크루즈 8일 가격: 1,258,875 원~ 기간: 출발 가능 기간 : 2018.06.17~2018.09.16 검색어: 미국
content final  인터파크 혜택 <div class="default-section special" id="divGoodsSpecialInfo"><h3>인터파크 혜택</h3><p><span style="FONT-FAMILY: "나눔고딕코딩",NanumGothicCoding,Sans-serif; COLOR: #000000; FONT-SIZE: 9pt"><strong><font color="#0075c8"><span style="COLOR: #000000">1. 프리미엄급 럭셔리 크루즈 선사, 프린세스크루즈 에메랄드프린세스호 승선~!</span><br/><span style="COLOR: #000000">2. 알래스카 천혜의 자연과 빙하를 눈 앞에서 볼 수 있는 일정~</span><br/><span style="COLOR: #000000">3. 각 기항지에서 다양한 자유 투어 (야생동물관찰, 개썰매체험, 헬기투어 등)</span></font></strong></span></p><p><span style="FONT-FAMILY: "나눔고딕코딩",NanumGothicCoding,Sans-serif; COLOR: #000000; FONT-SIZE: 9pt"><font color="#0075c8"><span style="COLOR: #ef007c">* 기항지에 도착 후에는 자유롭게 시간을 보낼 수 있습니다.</span><br/><span style="COLOR: #ef007c">* 기항지 선택관광은 희망자에 한하여 신청 가능합니다.</span><br/><span style="COLOR: #ef007c">* 선택 관광 미선택 시 불이익은 없으며, 미선택시 개별 자유일정으로 진행됩니다. (자세한 사항은 상세 일정표를 참조 해 주세요) </span></font></span></p><p><span style="F

SystemExit: 

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
